In [2]:
import json
import os
import ast
import csv
import io
from io import StringIO, BytesIO, TextIOWrapper
import gzip
from datetime import datetime, date
from s3_utils import *
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import ast
from datetime import timedelta
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn.model_selection import cross_val_score, ShuffleSplit, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import warnings
import sys
import time
from statsmodels.tsa.stattools import adfuller
from utils import *

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-0q1g3enq because the default path (/home/ubuntu/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:

kalman_df = pd.read_csv('./imputed_df_sleep_fatigue/kalman_impute_df_userid_1032.csv')
kalman_df

NameError: name 'pd' is not defined

In [6]:
mean_df = pd.read_csv('./imputed_df_sleep_fatigue/mean_impute_df_userid_1032.csv')

In [7]:
def stationary_test(series):
    result = adfuller(series, autolag='AIC')
    print(f'ADF Statistic: {result[0]}')
    print(f'n_lags: {result[1]}')
    print(f'p-value: {result[1]}')
    for key, value in result[4].items():
        print('Critial Values:')
        print(f'   {key}, {value}')  

stationary_test(kalman_df['answer_text'])


ADF Statistic: -0.5661920377856506
n_lags: 0.8784397304792488
p-value: 0.8784397304792488
Critial Values:
   1%, -3.454988209954765
Critial Values:
   5%, -2.8723857312734613
Critial Values:
   10%, -2.572549407997327


In [25]:
stationary_test(kalman_df['hr_average'])


ADF Statistic: -1.8575439563705518
n_lags: 0.35228960159955947
p-value: 0.35228960159955947
Critial Values:
   1%, -3.4548039258751206
Critial Values:
   5%, -2.872304928618605
Critial Values:
   10%, -2.5725063100137175


In [26]:
stationary_test(kalman_df['breath_average'])

ADF Statistic: -3.027035128956414
n_lags: 0.03244563713705504
p-value: 0.03244563713705504
Critial Values:
   1%, -3.4548957220044336
Critial Values:
   5%, -2.8723451788613157
Critial Values:
   10%, -2.572527778361272


In [27]:
stationary_test(kalman_df['rmssd'])


ADF Statistic: -1.5314649203273019
n_lags: 0.5178926730737616
p-value: 0.5178926730737616
Critial Values:
   1%, -3.4554613060274972
Critial Values:
   5%, -2.8725931472675046
Critial Values:
   10%, -2.5726600403359887


In [9]:
stationary_test(mean_df['answer_text'])
stationary_test(mean_df['rmssd'])
stationary_test(mean_df['breath_average'])
stationary_test(mean_df['hr_average'])

ADF Statistic: -1.4913584583172514
n_lags: 0.5377817724658496
p-value: 0.5377817724658496
Critial Values:
   1%, -3.4548957220044336
Critial Values:
   5%, -2.8723451788613157
Critial Values:
   10%, -2.572527778361272
ADF Statistic: -1.909395314682709
n_lags: 0.3276542487032547
p-value: 0.3276542487032547
Critial Values:
   1%, -3.455365238788105
Critial Values:
   5%, -2.8725510317187024
Critial Values:
   10%, -2.5726375763314966
ADF Statistic: -3.1196408211122173
n_lags: 0.025133413121681127
p-value: 0.025133413121681127
Critial Values:
   1%, -3.4548957220044336
Critial Values:
   5%, -2.8723451788613157
Critial Values:
   10%, -2.572527778361272
ADF Statistic: -2.5482957417772925
n_lags: 0.10416113432665464
p-value: 0.10416113432665464
Critial Values:
   1%, -3.4546223782586534
Critial Values:
   5%, -2.8722253212300277
Critial Values:
   10%, -2.5724638500216264


In [11]:
test_df = pd.read_csv('/domino/datasets/local/Bump/mice_user_42.csv')

In [ ]:
test_df

In [15]:
import torch
from torch.utils import data
import matplotlib.pyplot as plt
import argparse
import math
import seaborn as sns; sns.set()
import sys

import numpy as np
import pickle
import os
import random

from statsmodels.tsa.stattools import adfuller
class TNCDataset(data.Dataset):
    def __init__(self, x, mc_sample_size, window_size, augmentation, epsilon=3, state=None, adf=False):
        super(TNCDataset, self).__init__()
        self.time_series = x
        self.T = x.shape[-1]
        self.window_size = window_size
        self.sliding_gap = int(window_size*25.2)
        self.window_per_sample = (self.T-2*self.window_size)//self.sliding_gap
        self.mc_sample_size = mc_sample_size
        self.state = state
        self.augmentation = augmentation
        self.adf = adf
        if not self.adf:
            self.epsilon = epsilon
            self.delta = 5*window_size*epsilon

    def __len__(self):
        return len(self.time_series)*self.augmentation

    def __getitem__(self, ind):
        ind = ind%len(self.time_series)
        print(2*self.window_size)
        print(self.T-2*self.window_size)
        t = np.random.randint(2*self.window_size, self.T-2*self.window_size)
        
        x_t = self.time_series[ind][:,t-self.window_size//2:t+self.window_size//2]
        plt.savefig('./plots/%s_seasonal.png'%ind)
        X_close = self._find_neighours(self.time_series[ind], t)
        X_distant = self._find_non_neighours(self.time_series[ind], t)

        if self.state is None:
            y_t = -1
        else:
            y_t = torch.round(torch.mean(self.state[ind][t-self.window_size//2:t+self.window_size//2]))
        return x_t, X_close, X_distant, y_t

    def _find_neighours(self, x, t):
        T = self.time_series.shape[-1]
        if self.adf:
            gap = self.window_size
            corr = []
            for w_t in range(self.window_size,4*self.window_size, gap):
                try:
                    p_val = 0
                    for f in range(x.shape[-2]):
                        p = adfuller(np.array(x[f, max(0,t - w_t):min(x.shape[-1], t + w_t)].reshape(-1, )))[1]
                        p_val += 0.01 if math.isnan(p) else p
                    corr.append(p_val/x.shape[-2])
                except:
                    corr.append(0.6)
            self.epsilon = len(corr) if len(np.where(np.array(corr) >= 0.01)[0])==0 else (np.where(np.array(corr) >= 0.01)[0][0] + 1)
            self.delta = 5*self.epsilon*self.window_size

        ## Random from a Gaussian
        t_p = [int(t+np.random.randn()*self.epsilon*self.window_size) for _ in range(self.mc_sample_size)]
        t_p = [max(self.window_size//2+1,min(t_pp,T-self.window_size//2)) for t_pp in t_p]
        x_p = torch.stack([x[:, t_ind-self.window_size//2:t_ind+self.window_size//2] for t_ind in t_p])
        return x_p

    def _find_non_neighours(self, x, t):
        T = self.time_series.shape[-1]
        if t>T/2:
            t_n = np.random.randint(self.window_size//2, max((t - self.delta + 1), self.window_size//2+1), self.mc_sample_size)
        else:
            t_n = np.random.randint(min((t + self.delta), (T - self.window_size-1)), (T - self.window_size//2), self.mc_sample_size)
        x_n = torch.stack([x[:, t_ind-self.window_size//2:t_ind+self.window_size//2] for t_ind in t_n])

        if len(x_n)==0:
            rand_t = np.random.randint(0,self.window_size//5)
            if t > T / 2:
                x_n = x[:,rand_t:rand_t+self.window_size].unsqueeze(0)
            else:
                x_n = x[:, T - rand_t - self.window_size:T - rand_t].unsqueeze(0)
        return x_n

In [3]:
import pandas as pd
kalman_df = pd.read_csv('./imputed_df_sleep_fatigue/kalman_impute_df_userid_1032.csv')
kalman_df


,X,date,user_id_x,answer_text,hr_average,rmssd,breath_average,score,user_id_y,birth_date
0,0,2021-10-25,1032,0.524594,-0.148764,0.225188,4.683366,76.000000,1032,2022-05-25
1,1,2021-10-26,1032,1.039473,-0.252406,0.404343,3.218078,75.000000,1032,2022-05-25
2,2,2021-10-27,1032,0.524594,-0.518913,0.897018,2.078409,89.000000,1032,2022-05-25
3,3,2021-10-28,1032,1.039473,-0.082138,0.449131,2.078409,83.000000,1032,2022-05-25
4,4,2021-10-29,1032,0.524594,-0.128036,0.449131,1.427170,77.000000,1032,2022-05-25
...,...,...,...,...,...,...,...,...,...,...
271,271,2022-07-11,1032,1.039473,-0.011069,-0.043545,-0.038118,74.000000,1032,2022-05-25
272,272,2022-07-12,1032,1.039473,-0.692142,0.941807,-0.200928,88.000000,1032,2022-05-25
273,273,2022-07-13,1032,1.039473,-1.879579,1.792793,-1.177787,89.000000,1032,2022-05-25
274,274,2022-07-14,1032,1.039473,-0.973455,0.493920,-0.200928,90.000000,1032,2022-05-25


In [24]:
ds = kalman_df[['hr_average', 'rmssd']]
ds = ds.to_numpy()
ds = ds.T

In [23]:
mydataset = TNCDataset(ds, mc_sample_size=20, window_size=7, augmentation=1, adf=True)
mydataset[0]

14
262


IndexError: invalid index to scalar variable.